In [151]:
import pandas as pd
dataset = pd.read_csv("loan.csv")

In [152]:
nFeatures = len(dataset.columns) - 1
featuresName = list(dataset.columns)
featuresValues = dict()
for row in dataset.itertuples():
    for column,value in zip(featuresName,list(row[1:])):
        if featuresValues.get(column) == None:
            featuresValues[column] = [value]
        else:
            if value not in featuresValues[column]:
                featuresValues[column].append(value)        
featuresName = featuresName[:-1]
for s in featuresValues:
    print(s,"->",featuresValues[s])

reputation -> ['excellent', 'bad', 'good']
lenght -> [5, 3]
income -> ['high', 'modest']
y -> ['safe', 'risky']


In [153]:
class Node:
    def __init__(self):
        self.parent = None
        self.feature = None
        self.link = None
        self.childs = None
        self.y = None
    def next(self, value):
        pass
    def getRoute(self):
        if self.parent == None:
            if self.link == None:
                return []
            return [self.link]
        return [self.link] + self.parent.getRoute()
    def getFeaturesRoute(self):
        if self.parent == None: return []
        return [self.parent.feature] + self.parent.getFeaturesRoute()

class Tree:
    def __init__(self):
        self.levels = 0
        self.root = Node()
    def getLeaves(self, node=None):
        if node == None: node = self.root
        if node.childs == None:
            return [node]
        return self.__getLeavesNodes(node.childs)
    def __getLeavesNodes(self, nodes):
        output = []
        for node in nodes:
            output.extend(self.getLeaves(node))
        return output
    def eval(self, row, names):
        node = self.root
        while node.y == None:
            i = names.index(node.feature)
            value = row[i]
            for child in node.childs:
                if child.link == value:
                    node = child
        return node.y

In [154]:
tree = Tree()
choosenFeatures = []
Y = featuresValues["y"]
for level in range(len(featuresName)):
    leaves = tree.getLeaves()
    for l in leaves:
        remainingFeatures = []
        routeName = l.getFeaturesRoute()
        route = l.getRoute()
        for f in featuresName:
            if f not in routeName:
                remainingFeatures.append(f)
        temp = dict()
        for row in dataset.itertuples():
            check = True
            for fea,feaVal in zip(routeName,route):
                i = featuresName.index(fea)
                if feaVal != row[1:-1][i]:
                    check = False
            if check:
                for elem,name in zip(row[1:-1],featuresName):
                    if temp.get((name,elem,row[-1])) == None:
                        temp[(name,elem,row[-1])] = 1
                    else:
                        temp[(name,elem,row[-1])] += 1
        minErrorFeature = None
        for f in remainingFeatures:
            totCount = 0
            best = []
            for v in featuresValues[f]:
                maxValue = 0
                maxValueY = None
                for y in Y:
                    if (temp.get((f,v,y))) != None:
                        totCount += (temp.get((f,v,y)))
                        if temp.get((f,v,y)) > maxValue:
                            maxValue = temp.get((f,v,y))
                            maxValueY = y
                best.append((v,maxValue,maxValueY))
            good = 0
            for _, number, _ in best:
                good += number
            errors = totCount - good
            if minErrorFeature == None:
                minErrorFeature = (errors, f, best)
            else:
                if errors < minErrorFeature[0]:
                   minErrorFeature = (errors, f, best)
        l.feature = minErrorFeature[1]
        childs = []
        for value,_,y in minErrorFeature[2]:
            child = Node()
            childs.append(child)
            child.parent = l
            child.link = value
            child.y = y
            if level == len(featuresName):
                child.y = y
        l.childs = childs

In [159]:
errors = 0
predictions = 0
for row in dataset.itertuples():
    predictions += 1
    prediction = tree.eval(row[1:-1],featuresName)
    if prediction != row[-1]:
        errors += 1
print("accuracy=",1-errors/predictions)

accuracy= 0.75
